In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
!pip install patool
import patoolib
import scipy.io
import glob
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
drive.mount('/content/gdrive')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

     |████████████████████████████████| 77 kB 3.3 MB/s 
Mounted at /content/gdrive


In [35]:
sim_dir =  '/content/gdrive/My Drive/פרויקט מסכם/new_simulations/'

for rar in glob.glob(sim_dir + '*.rar'):
  patoolib.extract_archive(rar, outdir=sim_dir)

patool: Extracting /content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_ellipses.rar ...
patool: running /usr/bin/unrar x -- "/content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_ellipses.rar"
patool:     with cwd='/content/gdrive/My Drive/פרויקט מסכם/new_simulations/'
patool: ... /content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_ellipses.rar extracted to `/content/gdrive/My Drive/פרויקט מסכם/new_simulations/'.
patool: Extracting /content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_both.rar ...
patool: running /usr/bin/unrar x -- "/content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_both.rar"
patool:     with cwd='/content/gdrive/My Drive/פרויקט מסכם/new_simulations/'
patool: ... /content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_both.rar extracted to `/content/gdrive/My Drive/פרויקט מסכם/new_simulations/'.
patool: Extracting /content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_spheres.rar ...
patool: running /usr/bin/unrar x -- "/content/gdrive/My Driv

In [36]:
dataset = []
labels = []
i = 0
for file in glob.glob(sim_dir + '*/*.mat'):
  i += 1
  if i%100==0:
    print(i)
  mat = scipy.io.loadmat(file)['logY'].toarray()
  dataset.append(mat)
  if "homogenous" in file:
    labels.append(0)
  else: 
    labels.append(1)
dataset = np.real(np.array(dataset))
labels = np.array(labels)
np.save(sim_dir + "dataset.npy", dataset)
np.save(sim_dir + "labels.npy", labels)

# dataset = np.load('/content/gdrive/My Drive/פרויקט מסכם/new_simulations/dataset.npy')
# labels = np.load('/content/gdrive/My Drive/פרויקט מסכם/new_simulations/labels.npy')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900


In [38]:
print(dataset.shape)
print(labels.shape)

# for i in range(1,5):
#   print(x_train_norm[i])
#   print(t_train[i])

(5950, 25, 25)
(5950,)


In [39]:
def norm_data(data):
  output = []
  for sample in data: 
    sample = (sample - np.mean(sample))/np.std(sample)
    output.append(np.array(sample))
  return np.array(output)

input_size = 25*25
num_of_samples = dataset.shape[0]

perm = torch.randperm(num_of_samples)
dataset = dataset[perm]
labels = labels[perm]

x_train, x_val, x_test = dataset[:round(0.7*num_of_samples)], dataset[round(0.7*num_of_samples):round(0.9*num_of_samples)], dataset[round(0.9*num_of_samples):]
t_train, t_val, t_test = labels[:round(0.7*num_of_samples)],  labels[round(0.7*num_of_samples):round(0.9*num_of_samples)],  labels[round(0.9*num_of_samples):]

x_train_norm = norm_data(x_train)
x_val_norm = norm_data(x_val)
x_test_norm = norm_data(x_test)

x_train_norm = np.reshape(x_train_norm, (x_train_norm.shape[0], x_train_norm.shape[1]*x_train_norm.shape[2]))
x_val_norm = np.reshape(x_val_norm, (x_val_norm.shape[0], x_val_norm.shape[1]*x_val_norm.shape[2]))
x_test_norm = np.reshape(x_test_norm, (x_test_norm.shape[0], x_test_norm.shape[1]*x_test_norm.shape[2]))

train_loader = torch.utils.data.DataLoader(
    np.concatenate((t_train[:,None], x_train_norm),axis=1),
    batch_size=64, shuffle=True)

val_loader = torch.utils.data.DataLoader(
    np.concatenate((t_val[:,None], x_val_norm),axis=1),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    np.concatenate((t_test[:,None], x_test_norm),axis=1),
    batch_size=64, shuffle=True)

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=9, num_hidden=50):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, num_hidden)
        self.layer2 = nn.Linear(num_hidden, num_hidden//2)
        self.layer3 = nn.Linear(num_hidden//2, 2)
        self.num_hidden = num_hidden
        self.input_size = input_size
    def forward(self, x):
        x = x.reshape([-1, input_size])
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = F.log_softmax(x, dim=1)
        return x 

class CNN(nn.Module):
    def __init__(self, input_size, n_feature):
        super(CNN, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_feature, kernel_size=5, padding='same')
        self.conv2 = nn.Conv2d(n_feature, n_feature, kernel_size=5, padding='same')
        self.fc1 = nn.Linear(n_feature*6*6, 50)
        self.fc2 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.view(-1, self.n_feature*6*6)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [12]:
def get_accuracy(model, loader=train_loader):
    model.eval()
    loss = 0
    correct = 0
    pred_list = []
    true_list = []
    for ar in loader:
        data = ar[:,1:]
        label = ar[:,0]
        # send to device
        data, label = data.to(device), label.to(device)

        data = data.view(-1, 25*25)
        data = data.view(-1, 1, 25, 25)
        
        pred = model(data)
        loss += F.nll_loss(pred, label.long(), reduction='sum').item() # sum up batch loss                                                               
        pred = pred.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(label.data.view_as(pred)).cpu().sum().item()
    # print(f"correct = {correct}")
    # print(f"num_of_ars = {num_of_ars}")
    loss /= len(loader.dataset)
    accuracy = 100. * correct / len(loader.dataset)
    
    return loss, accuracy

def train(model, lr=0.01, max_iters=1000,num_epochs=6):
    model.train()
    train_accs, valid_accs = [], []
    epochs = []
    optimizer = optim.Adam(model.parameters(), lr=lr)
    n = 0 # the number of iterations
    iters, losses = [], []
    iters_sub = []

    for epoch in range(0, num_epochs):

        for batch_idx, ar in enumerate(train_loader):           
            data = ar[:,1:]
            label = ar[:,0]
            # send to device
            data, label = data.to(device), label.to(device)
            data = data.view(-1, 25*25)
            data = data.view(-1, 1, 25, 25)

            optimizer.zero_grad()
            pred = model(data)
            loss = F.nll_loss(pred, label.long())
            loss.backward()
            optimizer.step()

            iters.append(n)
            losses.append(loss)

            if batch_idx % 64 == 0: 
                
                iters_sub.append(n)
                train_loss, train_acc = get_accuracy(model, loader=train_loader)
                train_accs.append(train_acc)

                valid_loss, valid_acc = get_accuracy(model, loader=val_loader)
                valid_accs.append(valid_acc)
      
                print("Iter %d. [Val Acc %.0f%%] [Train Acc %.0f%%, Loss %f]" % (n, valid_acc, train_acc, train_loss))

            # increment the iteration number
            n += 1
            if n > max_iters:
                return iters, losses, iters_sub, train_accs, valid_accs
    return iters, losses, iters_sub, train_accs, valid_accs

In [14]:

model = CNN(input_size, 10)
model.to(device)

lr = 0.0001

iters, losses, iters_sub, train_accs, valid_accs = train(model.double(), lr=lr, max_iters=10000, num_epochs=15)

Iter 0. [Val Acc 12%] [Train Acc 11%, Loss 0.713430]
Iter 44. [Val Acc 88%] [Train Acc 89%, Loss 0.398231]
Iter 88. [Val Acc 88%] [Train Acc 89%, Loss 0.350297]
Iter 132. [Val Acc 88%] [Train Acc 89%, Loss 0.350346]
Iter 176. [Val Acc 88%] [Train Acc 89%, Loss 0.350065]
Iter 220. [Val Acc 88%] [Train Acc 89%, Loss 0.350441]
Iter 264. [Val Acc 88%] [Train Acc 89%, Loss 0.350843]
Iter 308. [Val Acc 88%] [Train Acc 89%, Loss 0.351156]
Iter 352. [Val Acc 88%] [Train Acc 89%, Loss 0.350174]
Iter 396. [Val Acc 88%] [Train Acc 89%, Loss 0.350161]
Iter 440. [Val Acc 88%] [Train Acc 89%, Loss 0.350296]
Iter 484. [Val Acc 88%] [Train Acc 89%, Loss 0.350213]
Iter 528. [Val Acc 88%] [Train Acc 89%, Loss 0.349887]
Iter 572. [Val Acc 88%] [Train Acc 89%, Loss 0.350938]
Iter 616. [Val Acc 88%] [Train Acc 89%, Loss 0.349866]


In [40]:
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
from torch.autograd import Variable
# from torchvision import datasets, transforms
# from capsnet import CapsNet
# from data_loader import Dataset
from tqdm import tqdm

USE_CUDA = True if torch.cuda.is_available() else False
BATCH_SIZE = 64
N_EPOCHS = 100
LEARNING_RATE = 0.01
MOMENTUM = 0.9
n_features = 10
'''
Config class to determine the parameters for capsule net
'''

class Config:
    def __init__(self):
        # CNN (cnn)
        self.cnn_in_channels = 1
        self.cnn_out_channels = 256
        self.cnn_kernel_size = 9

        # Primary Capsule (pc)
        self.pc_num_capsules = 8
        self.pc_in_channels = 256
        self.pc_out_channels = 32
        self.pc_kernel_size = 9
        self.pc_num_routes = 32 * 5 * 5

        # Digit Capsule (dc)
        self.dc_num_capsules = 2
        self.dc_num_routes = 32 * 5 * 5
        self.dc_in_channels = 8
        self.dc_out_channels = 16

        # Decoder
        self.input_width = 25
        self.input_height = 25

from torch.autograd import Variable

USE_CUDA = True if torch.cuda.is_available() else False

class ConvLayer(nn.Module):
    def __init__(self, in_channels=1, out_channels=256, kernel_size=9):
        super(ConvLayer, self).__init__()

        self.conv = nn.Conv2d(in_channels=in_channels,
                              out_channels=out_channels,
                              kernel_size=kernel_size,
                              stride=1
                              )

    def forward(self, x):
        return F.relu(self.conv(x))


class PrimaryCaps(nn.Module):
    def __init__(self, num_capsules=8, in_channels=256, out_channels=32, kernel_size=9, num_routes=32 * 6 * 6):
        super(PrimaryCaps, self).__init__()
        self.num_routes = num_routes
        self.capsules = nn.ModuleList([
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=2, padding=0)
            for _ in range(num_capsules)])

    def forward(self, x):
        u = [capsule(x) for capsule in self.capsules]
        u = torch.stack(u, dim=1)
        u = u.view(x.size(0), self.num_routes, -1)
        return self.squash(u)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm * input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor


class DigitCaps(nn.Module):
    def __init__(self, num_capsules=10, num_routes=32 * 6 * 6, in_channels=8, out_channels=16):
        super(DigitCaps, self).__init__()

        self.in_channels = in_channels
        self.num_routes = num_routes
        self.num_capsules = num_capsules

        self.W = nn.Parameter(torch.randn(1, num_routes, num_capsules, out_channels, in_channels))

    def forward(self, x):
        batch_size = x.size(0)
        x = torch.stack([x] * self.num_capsules, dim=2).unsqueeze(4)

        W = torch.cat([self.W] * batch_size, dim=0)
        u_hat = torch.matmul(W, x)

        b_ij = Variable(torch.zeros(1, self.num_routes, self.num_capsules, 1))
        if USE_CUDA:
            b_ij = b_ij.cuda()

        num_iterations = 3
        for iteration in range(num_iterations):
            c_ij = F.softmax(b_ij, dim=1)
            c_ij = torch.cat([c_ij] * batch_size, dim=0).unsqueeze(4)

            s_j = (c_ij * u_hat).sum(dim=1, keepdim=True)
            v_j = self.squash(s_j)

            if iteration < num_iterations - 1:
                a_ij = torch.matmul(u_hat.transpose(3, 4), torch.cat([v_j] * self.num_routes, dim=1))
                b_ij = b_ij + a_ij.squeeze(4).mean(dim=0, keepdim=True)

        return v_j.squeeze(1)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm * input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor


class Decoder(nn.Module):
    def __init__(self, input_width=28, input_height=28, input_channel=1):
        super(Decoder, self).__init__()
        self.input_width = input_width
        self.input_height = input_height
        self.input_channel = input_channel
        self.reconstraction_layers = nn.Sequential(
            nn.Linear(16 * 2, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, self.input_height * self.input_width * self.input_channel),
            nn.Sigmoid()
        )

    def forward(self, x, data):
        classes = torch.sqrt((x ** 2).sum(2))
        classes = F.softmax(classes, dim=0)

        _, max_length_indices = classes.max(dim=1)
        masked = Variable(torch.sparse.torch.eye(2))
        if USE_CUDA:
            masked = masked.cuda()
        masked = masked.index_select(dim=0, index=Variable(max_length_indices.squeeze(1).data))
        t = (x * masked[:, :, None, None]).view(x.size(0), -1)
        reconstructions = self.reconstraction_layers(t)
        reconstructions = reconstructions.view(-1, self.input_channel, self.input_width, self.input_height)
        return reconstructions, masked


class CapsNet(nn.Module):
    def __init__(self, config=None):
        super(CapsNet, self).__init__()
        if config:
            self.conv_layer = ConvLayer(config.cnn_in_channels, config.cnn_out_channels, config.cnn_kernel_size)
            self.primary_capsules = PrimaryCaps(config.pc_num_capsules, config.pc_in_channels, config.pc_out_channels,
                                                config.pc_kernel_size, config.pc_num_routes)
            self.digit_capsules = DigitCaps(config.dc_num_capsules, config.dc_num_routes, config.dc_in_channels,
                                            config.dc_out_channels)
            self.decoder = Decoder(config.input_width, config.input_height, config.cnn_in_channels)
        else:
            self.conv_layer = ConvLayer()
            self.primary_capsules = PrimaryCaps()
            self.digit_capsules = DigitCaps()
            self.decoder = Decoder()

        self.mse_loss = nn.MSELoss()

    def forward(self, data):
        output = self.digit_capsules(self.primary_capsules(self.conv_layer(data)))
        reconstructions, masked = self.decoder(output, data)
        return output, reconstructions, masked

    def loss(self, data, x, target, reconstructions):
        return self.margin_loss(x, target) + self.reconstruction_loss(data, reconstructions)

    def margin_loss(self, x, labels, size_average=True):
        batch_size = x.size(0)

        v_c = torch.sqrt((x ** 2).sum(dim=2, keepdim=True))

        left = F.relu(0.9 - v_c).view(batch_size, -1)
        right = F.relu(v_c - 0.1).view(batch_size, -1)

        loss = labels * left + 0.5 * (1.0 - labels) * right
        loss = loss.sum(dim=1).mean()

        return loss

    def reconstruction_loss(self, data, reconstructions):
        loss = self.mse_loss(reconstructions.view(reconstructions.size(0), -1), data.view(reconstructions.size(0), -1))
        return loss * 0.0005

# def new_margin_loss(x, labels, size_average=True):
#         batch_size = x.size(0)

#         v_c = torch.sqrt((x ** 2).sum(dim=2, keepdim=True))

#         left = F.relu(0.9 - v_c).view(batch_size, -1)
#         right = F.relu(v_c - 0.1).view(batch_size, -1)
#         loss = labels * left + 0.5 * (1.0 - labels) * right
#         loss = loss.sum(dim=1).mean()

#         return loss

def train_capsnet(model, optimizer, train_loader, epoch):
    capsule_net = model
    capsule_net.train()
    n_batch = len(list(enumerate(train_loader)))
    total_loss = 0
    for batch_id, ar in enumerate(train_loader):
        data = ar[:,1:].type(torch.double)
        target = ar[:,0].type(torch.int64)
        target = torch.sparse.torch.eye(2).index_select(dim=0, index=target)
        data, target = Variable(data), Variable(target)

        data = data.view(-1, 25*25)
        data = data.view(-1, 1, 25, 25)

        if USE_CUDA:
            data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output, reconstructions, masked = capsule_net(data)
        loss = capsule_net.loss(data, output, target, reconstructions)
        loss.backward()
        optimizer.step()
        correct = sum(np.argmax(masked.data.cpu().numpy(), 1) == np.argmax(target.data.cpu().numpy(), 1))
        train_loss = loss.item()
        total_loss += train_loss
        if batch_id % 100 == 0:
            current_acc, current_loss = get_accuracy_capsnet(capsule_net, train_loader)
            # tqdm.write(
            #     "Epoch: [{}/{}], train accuracy: {:.6f}%, loss: {:.6f}".format(e, N_EPOCHS, valid_acc,
            #                                                               valid_loss / len(val_loader)))
            tqdm.write("Epoch: [{}/{}], Batch: [{}/{}], train accuracy: {:.6f}%, loss: {:.6f}".format(
                epoch,
                N_EPOCHS,
                batch_id + 1,
                n_batch,
                current_acc,
                current_loss
                ))
    tqdm.write('Epoch: [{}/{}], train loss: {:.6f}'.format(epoch,N_EPOCHS,total_loss / len(train_loader.dataset)))


def get_accuracy_capsnet(capsule_net, loader):
    capsule_net.eval()
    final_loss = 0
    correct = 0

    pred_list = []
    true_list = []
    for batch_id, ar in enumerate(loader):
        data = ar[:,1:].type(torch.double)
        target = ar[:,0].type(torch.int64)
        target = torch.sparse.torch.eye(2).index_select(dim=0, index=target)
        data, target = Variable(data), Variable(target)

        data = data.view(-1, 25*25)
        data = data.view(-1, 1, 25, 25)

        if USE_CUDA:
            data, target = data.cuda(), target.cuda()

        output, reconstructions, masked = capsule_net(data)
        loss = capsule_net.loss(data, output, target, reconstructions)

        final_loss += loss.item()
        correct += sum(np.argmax(masked.data.cpu().numpy(), 1) ==
                       np.argmax(target.data.cpu().numpy(), 1))
        
        for y in np.argmax(masked.data.cpu().numpy(), 1):
          pred_list.append(y)

        for y in np.argmax(target.data.cpu().numpy(), 1):  
          true_list.append(y)

    # conf_mat = confusion_matrix(true_list, pred_list, labels=list(range(0,9))+list(range(10,25)))  
    acc = correct * 100 / len(loader.dataset)
    
    return acc, final_loss

In [ ]:
torch.manual_seed(1)
config = Config()

capsule_net = CapsNet(config)
capsule_net = torch.nn.DataParallel(capsule_net)
if USE_CUDA:
    capsule_net = capsule_net.cuda()
capsule_net = capsule_net.module.double()

optimizer = torch.optim.Adam(capsule_net.parameters())
# capsule_net.margin_loss = new_margin_loss
for e in range(1, N_EPOCHS + 1):
    train_capsnet(capsule_net, optimizer, train_loader, e)
    valid_acc, valid_loss = get_accuracy_capsnet(capsule_net, val_loader)
    tqdm.write(
        "Epoch: [{}/{}], validation accuracy: {:.6f}%, loss: {:.6f}".format(e, N_EPOCHS, valid_acc,
                                                                  valid_loss / len(val_loader)))

torch.save(capsule_net.state_dict(),  "/content/gdrive/My Drive/פרויקט מסכם/new_simulations/capsnet.pt")

Epoch: [1/100], Batch: [1/66], train accuracy: 58.103241%, loss: 6.611827
Epoch: [1/100], train loss: 0.001672
Epoch: [1/100], validation accuracy: 51.008403%, loss: 0.088243
Epoch: [2/100], Batch: [1/66], train accuracy: 51.308523%, loss: 6.247734
Epoch: [2/100], train loss: 0.001468
Epoch: [2/100], validation accuracy: 50.672269%, loss: 0.086947
Epoch: [3/100], Batch: [1/66], train accuracy: 52.460984%, loss: 5.847939
Epoch: [3/100], train loss: 0.001412
Epoch: [3/100], validation accuracy: 52.184874%, loss: 0.088374
Epoch: [4/100], Batch: [1/66], train accuracy: 52.244898%, loss: 5.985426
Epoch: [4/100], train loss: 0.001411
Epoch: [4/100], validation accuracy: 54.201681%, loss: 0.086398
Epoch: [5/100], Batch: [1/66], train accuracy: 56.158463%, loss: 5.890804
Epoch: [5/100], train loss: 0.001413
Epoch: [5/100], validation accuracy: 54.621849%, loss: 0.086056
Epoch: [6/100], Batch: [1/66], train accuracy: 57.022809%, loss: 5.862510
Epoch: [6/100], train loss: 0.001410
Epoch: [6/100]